# 🪞 Módulo 6 — NumPy: Copias y Vistas

NumPy optimiza la memoria creando **vistas** de arrays cuando es posible.

Esto puede provocar comportamientos inesperados si no conocemos:

- Cuándo se crea una copia real
- Cuándo solo se crea una *vista* de los mismos datos
- Cómo afectan los slicing
- Diferencias entre `.view()` y `.copy()`

Este notebook es esencial para evitar bugs en ETL, ML, pipelines y procesado numérico.

---
## 1️⃣ Slicing devuelve **vistas**, no copias

Ejemplo clásico:

In [1]:
import numpy as np

x = np.arange(10)
v = x[2:7]
v[0] = 999
x, v

(array([  0,   1, 999,   3,   4,   5,   6,   7,   8,   9]),
 array([999,   3,   4,   5,   6]))

➡️ Al modificar `v`, también cambia `x` porque comparten memoria.

---
## 2️⃣ Cómo comprobar si dos arrays comparten memoria

NumPy lo indica con `np.shares_memory`:

In [2]:
np.shares_memory(x, v)

True

---
## 3️⃣ `.copy()` crea un nuevo array completamente independiente


In [3]:
x = np.arange(10)
c = x[2:7].copy()
c[0] = 111
x, c, np.shares_memory(x, c)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([111,   3,   4,   5,   6]),
 False)

✔️ Ahora `x` no se ve afectado.

---
## 4️⃣ `.view()` crea una nueva *vista* del mismo bloque de memoria

Muy útil para reinterpretar datos sin copiarlos.

In [4]:
a = np.arange(5)
v = a.view()
v[1] = 777
a, v

(array([  0, 777,   2,   3,   4]), array([  0, 777,   2,   3,   4]))

---
## 5️⃣ Casos donde slicing SÍ produce copia

Si el array no es contiguo en memoria, NumPy crea una copia.

Ejemplo: transpuesta de una matriz y luego slicing:

In [ ]:
m = np.arange(16).reshape(4,4)
mt = m.T
s = mt[:2, :2]

#parece que este caso no produce copia, pues devuelve True ambas pruebas
np.shares_memory(m, s), np.shares_memory(mt, s)

(True, True)

In [ ]:
#buscar un caso que sí produzca copia, aún sin haber llamado a .copy()
a = np.arange(10)
b = a[[0, 2, 5]] # copia implicita

np.shares_memory(a, b)

False

Esto depende de la implementación y layout interno.

---
## 6️⃣ Ejemplo práctico importante: evitar mutaciones accidentales

Procesado de imágenes o matrices donde queremos aislar datos:

In [ ]:
img = np.arange(100).reshape(10,10)
region = img[2:5, 2:5]  # vista
region[:] = 999
img

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9],
       [ 10,  11,  12,  13,  14,  15,  16,  17,  18,  19],
       [ 20,  21, 999, 999, 999,  25,  26,  27,  28,  29],
       [ 30,  31, 999, 999, 999,  35,  36,  37,  38,  39],
       [ 40,  41, 999, 999, 999,  45,  46,  47,  48,  49],
       [ 50,  51,  52,  53,  54,  55,  56,  57,  58,  59],
       [ 60,  61,  62,  63,  64,  65,  66,  67,  68,  69],
       [ 70,  71,  72,  73,  74,  75,  76,  77,  78,  79],
       [ 80,  81,  82,  83,  84,  85,  86,  87,  88,  89],
       [ 90,  91,  92,  93,  94,  95,  96,  97,  98,  99]])

La región modifica la imagen original porque es una vista.

Solución:
```python
region = img[2:5, 2:5].copy()
```

In [11]:
img2 = np.arange(100).reshape(10,10)
region2 = img2[2:5, 2:5].copy()  # copia
region2[:] = 999
img2

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])

---
## 7️⃣ Ejercicio práctico

Dado el array:
```python
x = np.arange(1,13).reshape(3,4)
```

### 🧩 Objetivos
1. Obtén una vista de la segunda fila
2. Modifica la vista y comprueba si cambia el original
3. Crea una copia de esa fila y modifica la copia
4. Comprueba memoria compartida en ambos casos

Escribe tu solución abajo:

In [ ]:
x = np.arange(1,13).reshape(3,4)
print("Datos ini:", x)

fila_vista = x[1]
print("1. Fila:", fila_vista)

fila_vista[1] = 999
print("2. Vista fila:", fila_vista)
print("2. Datos:", x)

fila_copia = x[1].copy()
fila_copia[0] = 999
print("3. Copia fila:", fila_copia)
print("3. Datos:", x)

print("4. Mem compartida datos-vista:", np.shares_memory(x, fila_vista))
print("4. Mem compartida datos-copia:", np.shares_memory(x, fila_copia))
print("4. Mem compartida vista-copia:", np.shares_memory(fila_vista, fila_copia))

print("Datos fin:", x)



Datos ini: [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
1. Fila: [5 6 7 8]
2. Vista fila: [  5 999   7   8]
2. Datos: [[  1   2   3   4]
 [  5 999   7   8]
 [  9  10  11  12]]
3. Copia fila: [999 999   7   8]
3. Datos: [[  1   2   3   4]
 [  5 999   7   8]
 [  9  10  11  12]]
4. Mem compartida datos-vista: True
4. Mem compartida datos-copia: False
4. Mem compartida vista-copia: False
Datos fin: [[  1   2   3   4]
 [  5 999   7   8]
 [  9  10  11  12]]


---
## ✅ Solución (oculta)

    
<details>
<summary>Mostrar solución</summary>

```python
x = np.arange(1,13).reshape(3,4)

# 1. Vista
v = x[1]

# 2. Modificar vista
v[0] = 999

# 3. Copia
c = x[1].copy()
c[1] = 555

# 4. Comprobaciones
np.shares_memory(x, v), np.shares_memory(x, c)
```
</details>